In [40]:
import lxml
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from requests import request
import json
import urllib
from lxml import etree, html as lhtml
import re
import gzip
import codecs
from time import sleep
import sys
import os
from multiprocessing.dummy import Pool, Queue
import pandas as pd
from itertools import chain
from typing import Generator, Dict, Any

In [2]:
base_url = "https://shop.relod.ru/catalog-products/4577/?sort=PROPERTY_RATING&order=desc&PAGEN_1="
pages_url = []

In [3]:
for number_of_page in tqdm(range(1, 251)):
    curr_url = base_url + str(number_of_page)
    html_text = request('GET', curr_url)
    tree = lhtml.fromstring(html_text.text)
    pages_url.append(tree.xpath('//div[@class ="bxr-element-name"]/a/@href'))

100%|██████████| 250/250 [00:34<00:00,  7.26it/s]


In [4]:
flat_list = [item for sublist in pages_url for item in sublist]

In [5]:
len(set(flat_list))

4962

In [6]:
while len(set(flat_list)) < 5000:
    number_of_page += 1
    curr_url = base_url + str(number_of_page)
    html_text = request('GET', curr_url)
    tree = lhtml.fromstring(html_text.text)
    obj = tree.xpath('//div[@class ="bxr-element-name"]/a/@href')
    pages_url.append(tree.xpath('//div[@class ="bxr-element-name"]/a/@href'))
    flat_list = [item for sublist in pages_url for item in sublist]
flat_list = list(set(flat_list))[:5000]

In [7]:
len(set(flat_list))

5000

In [8]:
assert(len(flat_list) == len(set(flat_list)))

1) поле "URL" (ссылка на книгу);

2) поле "Название";

3) поле "Иллюстрации" (список из ссылок на изображения, предоставленные к книге);

4) поле "Метки" (список меток, всего их 4 вида: Sale, New, Хит, ПТВ);

5) поля "Оценка" и "Число голосов" (количество звезд у книги и количество пользователей, оценивших книгу);

6) поле "Наличие" (товар в наличии или доступен под заказ);

7) поле "Цена";

8) поле "Цена (скидка)" (цена уцененного товара, ее можно увидеть на сайте проставив галочку);

9) все поля из таблицы;

10) поле "Описание".

In [15]:
base_book_url = "https://shop.relod.ru"
regex_float = re.compile('"PRICE" : ([0-9]*\.[0-9])')
regex_int = re.compile('"PRICE" : ([0-9]*)')
regex_symbols = re.compile(r'[\n\r\t]')
def process_page(url, n_attempts=5, t_sleep=1):
    json_base = {}
    markers = []
    curr_url = base_book_url + url
    json_base['url'] = curr_url #url книги
    
    for i in range(n_attempts):
        html_test = request('GET', curr_url)
        if html_test.status_code == 200:
            break
        else:
            sleep(t_sleep)
    if html_test.status_code != 200:
        raise "Request GET Problem"
    tree = lhtml.fromstring(html_test.text)
    soup = BeautifulSoup(html_test.text, 'html.parser')
    
    nickname = tree.xpath('//div[contains(@class,"bxr-cloud-all")]/h1')
    for obj in nickname:
        json_base['Название'] = obj.text # Название книги
        
    star_rating = tree.xpath('//meta[contains(@itemprop, "ratingValue")]')
    for obj in star_rating:
        json_base['Оценка'] = obj.get('content') #звездочки
        
    votes_rating = tree.xpath('//meta[contains(@itemprop, "ratingCount")]')
    for obj in votes_rating:
        json_base['Число голосов'] = obj.get('content') # число голосов
        
    availability = tree.xpath('//div[@itemprop="availability"]')
    for obj in availability:
        json_base['Наличие'] = obj.text # наличие
        
    price_obj = tree.xpath('//div[contains(@class, "bxr-detail-price-line")]/meta[@itemprop="price"]')
    for obj in price_obj:
        json_base['Цена'] = obj.get('content') #цена
        
    if len(tree.xpath('//div[contains(@class, "ptv-block")]')) != 0: #есть уцененная версия книги
        checker_new = tree.xpath('//script') 
        for obj_new in checker_new:
            if obj_new.text is not None:
                if "var ptvSettings" in obj_new.text: # получили уценненую стоимость
                    float_price = regex_float.findall(obj_new.text)
                    if len(float_price) != 0:
                        json_base['Цена (скидка)'] = float(float_price[0])
                    else:
                        json_base['Цена (скидка)'] = int(regex_int.findall(obj_new.text)[0])
                    break
                    
    ptv_marker = tree.xpath('//a[contains(@class, "bxr-marker-ptv")]') #маркер ptv
    for obj in ptv_marker:
        markers.append(obj.text)
        
    other_marker = tree.xpath('//div[contains(@class , "bxr-ribbon-marker-vertical")]/span/i') #остальные маркеры
    for marker in other_marker:
            markers.append(marker.text)
            
    if len(markers):
        json_base['Метки'] = markers
        
    json_base['Иллюстрации'] = tree.xpath('//div[contains(@class, "bxr-element-slider-main")]/a/attribute::href')
    
    properties_names = soup.find_all('td', class_= "bxr-props-name")
    properties_data = soup.find_all('td', class_ = "bxr-props-data")
    
    for i in range(len(properties_names)):
        json_base[properties_names[i].text.replace("\n", "")] = properties_data[i].text.replace("\n", "").replace("(сайт издательства)", '') #записали свойства
        
    if soup.find('div', class_ = "bxr-detail") != None:
        json_base['Описание'] = re.sub(' +', ' ', regex_symbols.sub(" ", soup.find('div', class_ = "bxr-detail").text)).replace("\xa0", "") #Получили описание
        
    return json_base

In [17]:
queue = Queue() # очередь ссылок на книги
for obj in flat_list:
    queue.put(obj)

In [18]:
def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                url = queue.get()
                record = process_page(url)
            except Exception as e:
                print(url, file=sys.stderr)
                print(e, file=sys.stderr)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

100%|██████████| 5000/5000 [43:24<00:00,  1.92it/s]


In [36]:
def records_reader(dirname: str) -> Generator[Dict[str, Any], None, None]:
    path = './' + dirname
    obj = os.scandir(path)
    fhs = []
    for entry in obj : 
        if entry.is_file():
            try:
                with gzip.open('parse_results/' + entry.name, mode='rb') as f:
                    f = codecs.getreader('utf8')(f)
                    text = f.read().split('\n')
                    fhs.append([json.loads(i) for i in text[:-1]])
            except:
                print("Problem with file " + entry.name, file = sys.stderr)
    return chain(*fhs)

In [37]:
df = pd.DataFrame(records_reader('parse_results'))
df.to_csv('hw_3.csv', index=False)

In [39]:
df

,url,Название,Наличие,Цена,Иллюстрации,ISBN,Издатель,Автор,Язык,Возрастные ограничения,...,Страна происхождения,Цена (скидка),Метки,Носитель,Тип продукта,Оценка,Число голосов,Рейтинг,Обзор/Ролик,Издание
0,https://shop.relod.ru/catalog-products/the_com...,The Comfort of Saturdays,Под заказ,989,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780349120553,Little Brown,Alexander McCall Smith,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://shop.relod.ru/catalog-products/thud/,Thud!,Под заказ,1113,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780552167697,Corgi,Pratchett Terry,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://shop.relod.ru/catalog-products/the_hou...,The House That Groaned,Под заказ,1452,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780224086813,Vintage,Fransman Karrie,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://shop.relod.ru/catalog-products/the_rin...,The Ring,Под заказ,989,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9781848664586,Quercus,Saviano Roberto,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://shop.relod.ru/catalog-products/the_sto...,The Stolen Ones,Под заказ,737,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780340978665,Hodder & Stoughton,Keenan Shy,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,https://shop.relod.ru/catalog-products/the_nov...,The November Man,В наличии,392,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9781455558759,Little Brown,Granger Bill,English,16+,...,США,313.6,[ПТВ],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,https://shop.relod.ru/catalog-products/vanity_...,Vanity Fair,В наличии,508,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780198727712,Oxford University Press,Thackeray William Makepeace,English,16+,...,Соединенное Королевство,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,https://shop.relod.ru/catalog-products/to_the_...,To the Last City,Под заказ,968,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9780099437239,Vintage,Colin Thubron,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,https://shop.relod.ru/catalog-products/the_hol...,The Hollow Men,Под заказ,NaN,[//opt-1458870.ssl.1c-bitrix-cdn.ru/upload/ibl...,9781473617629,Mulholland Books,Rob McCarthy,English,16+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
